In [1]:
import requests
import re
import bs4
import pandas as pd

In [2]:
# Set up
# 1. Go to SEC EDGAR Company Search
# 2. Type in Filing Type (ex. 10-K, 100 Entries, Search All)
# 3. Paste the URL into this code
# 4. Extract Accession Codes to 

URL = input('EDGAR Search Result URL')

DAL = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0000027904&owner=exclude&count=100'

EDGAR Search Result URLhttps://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0000027904&owner=exclude&count=100


In [3]:
def get_next_page(someURL, start):
    # check if the next page exists based on the inputted URL
    someRes = requests.get(someURL)
    someSoup = bs4.BeautifulSoup(someRes.text,'html.parser')
    some_nextButton = someSoup.find_all('input', type = 'button', value = "Next 100", limit = 1)
    
    if not some_nextButton:
        start = start + 100
        newURL = someURL + '&start=' + str(start)
        return newURL
    
    # if it exists return the newURL
    newURL = someURL + '&start=' + str(start)
    
    return newURL

In [4]:
def get_filings(somePage):
    someRes = requests.get(somePage)
    someSoup = bs4.BeautifulSoup(someRes.text,'html.parser')
    
    descriptions = []
    
    for i in someSoup.find_all('tr'):
        for j in i.find_all('a', attrs = {'href' : re.compile('Archives')}):
            if '10-Q' in i.text or '10-K' in i.text:
                descriptions.append(i.text)
                
    # Clean Up
    filingType = []
    for i in descriptions:
        if '10-Q' in i:
            filingType.append('10-Q')
        if '10-K' in i:
            filingType.append('10-K')
    
    return filingType

In [5]:
def get_links(somePage):
    someRes = requests.get(somePage)
    someSoup = bs4.BeautifulSoup(someRes.text,'html.parser')
    
    links = []
    for i in someSoup.find_all('tr'):
        for j in i.find_all('a', attrs = {'href' : re.compile('Archives')}):
            if '10-Q' in i.text or '10-K' in i.text:
                links.append(j.get('href'))
    
    return links

In [6]:
start = []
num = 100
while num < 2000:
    start.append(num)
    num += 100

In [7]:
pages = []
for i in start:
    pages.append(get_next_page(URL,i))

In [8]:
real_pages = []

for i in pages:
    someRes = requests.get(i)
    someSoup = bs4.BeautifulSoup(someRes.text,'html.parser')
    some_nextButton = someSoup.find_all('input', type = 'button', value = "Next 100", limit = 1)
    
    if not some_nextButton:
        pass
    else:
        real_pages.append(i)

In [9]:
all_filings = []
for i in real_pages:
    all_filings.append(get_filings(i))

In [10]:
all_links = []
for i in real_pages:
    all_links.append(get_links(i))

In [11]:
filings = []
for i in all_filings:
    filings.extend(i)
    
links =[]

for i in all_links:
    links.extend(i)
    
df = pd.DataFrame({'Filings':filings,'Links':links})

In [12]:
df

,Filings,Links
0,10-Q,/Archives/edgar/data/27904/000002790418000020/...
1,10-Q,/Archives/edgar/data/27904/000002790418000016/...
2,10-Q,/Archives/edgar/data/27904/000002790418000013/...
3,10-K,/Archives/edgar/data/27904/000002790418000006/...
4,10-Q,/Archives/edgar/data/27904/000002790417000017/...
...,...,...
86,10-Q,/Archives/edgar/data/27904/0000950144-00-00678...
87,10-Q,/Archives/edgar/data/27904/0000950144-00-00217...
88,10-Q,/Archives/edgar/data/27904/0000950144-99-01327...
89,10-K,/Archives/edgar/data/27904/0000950144-99-01146...
